### 1. Crawl link bán nhà từ trang chợ tốt

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time



arr = []

for page in range(0,85):
        # Khởi tạo trình duyệt
    options = webdriver.ChromeOptions()
    #options.add_argument("--headless")  # Chạy không hiển thị trình duyệt (nếu cần)
    # options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    # time.sleep(30) 
    page = str(page + 1)
    link = "https://www.nhatot.com/mua-ban-dat-ha-noi?page=" + page
    driver.get(link)
    time.sleep(3)  # Chờ trang web tải dữ liệu (có thể điều chỉnh thời gian tùy thuộc vào tốc độ mạng)

    # Tìm các thẻ chứa link của các bài đăng bằng XPath
    elements = driver.find_elements(By.XPATH, "//a[@class='crd7gu7']")

    for element in elements:
        post_link = element.get_attribute('href')
        arr.append(post_link)
        print(post_link)
    
    # Đóng trình duyệt
    driver.quit()



# Lưu kết quả vào DataFrame
df = pd.DataFrame(arr, columns=['links'])
print(df)



https://www.nhatot.com/mua-ban-dat-huyen-thuong-tin-ha-noi/120093115.htm#px=SR-stickyad-[PO-1][PL-top]
https://www.nhatot.com/mua-ban-dat-huyen-soc-son-ha-noi/120671648.htm#px=SR-stickyad-[PO-2][PL-top]
https://www.nhatot.com/mua-ban-dat-huyen-thach-that-ha-noi/120705355.htm
https://www.nhatot.com/mua-ban-dat-huyen-thach-that-ha-noi/120705212.htm
https://www.nhatot.com/mua-ban-dat-quan-long-bien-ha-noi/120704994.htm
https://www.nhatot.com/mua-ban-dat-huyen-dong-anh-ha-noi/120704955.htm#px=SR-special_display_ad-[PO-6][PL-default]
https://www.nhatot.com/mua-ban-dat-quan-long-bien-ha-noi/119195838.htm
https://www.nhatot.com/mua-ban-dat-quan-long-bien-ha-noi/117607313.htm
https://www.nhatot.com/mua-ban-dat-quan-dong-da-ha-noi/120704823.htm
https://www.nhatot.com/mua-ban-dat-quan-hoang-mai-ha-noi/119844067.htm#px=SR-special_display_ad-[PO-10][PL-default]
https://www.nhatot.com/mua-ban-dat-quan-dong-da-ha-noi/120704420.htm
https://www.nhatot.com/mua-ban-dat-quan-hai-ba-trung-ha-noi/120704355

In [22]:
df.count()

links    1704
dtype: int64

In [23]:
# Xuất file csv
df.to_csv('datthocu_0_85.csv',index=True)
print("done")

done


### 2. Crawl dữ liệu chi tiết về nhà ở từ link bán nhà 

**2.1 Nhà ở**

In [22]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

k=0

# Đọc danh sách URL từ file CSV
input_file = 'nha_o_0-100.csv'  
urls_df = pd.read_csv(input_file)

# Hàm lấy thông tin từ trang
def get_text_or_none(driver, by, identifier):
    try:
        element = driver.find_element(by, identifier)
        return element.text
    except Exception as e:
        return None

# Danh sách để lưu dữ liệu từ mỗi trang
all_data = []

# Vòng lặp qua từng URL để lấy dữ liệu
for url in urls_df['links']:
    # Khởi tạo trình duyệt cho từng lần lặp
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    driver.get(url)
    # time.sleep(2)  # Chờ trang tải xong

    # Nhấn vào nút "Xem thêm" nếu có
    try:
        xem_them_button = driver.find_element(By.XPATH, '//button[contains(@class, "styles_button__SVZnw") and contains(text(), "Xem thêm")]')
        driver.execute_script("arguments[0].click();", xem_them_button)
        # time.sleep(1)  # Đợi một chút để phần còn lại của trang tải xong
    except Exception as e:
        print("Không tìm thấy nút 'Xem thêm' hoặc lỗi khi nhấn:", e)

    # Lấy thông tin từ trang
    data = {
        'Link': url,
        'Địa chỉ': get_text_or_none(driver, By.CLASS_NAME, 'bwq0cbs.flex-1'),
        'Giá tổng': get_text_or_none(driver, By.CSS_SELECTOR, 'b.pyhk1dv'),
        'Giá/m²': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="price_m2"]'),
        'Số phòng ngủ': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="rooms"]'),
        'Số phòng vệ sinh': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="toilets"]'),
        'Tổng số tầng': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="floors"]'),
        'Giấy tờ pháp lý': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="property_legal_document"]'),
        'Loại hình nhà ở': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="house_type"]'),
        'Chiều ngang': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="width"]'),
        'Chiều dài': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="length"]'),
        'Diện tích sử dụng': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="living_size"]'),
        'Diện tích đất': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="size"]'),
        'Hướng cửa chính': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="direction"]'),
        'Tình trạng nội thất': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="furnishing_sell"]')
    }

    # Thêm dữ liệu vào danh sách
    all_data.append(data)
    print(f"Đã lấy dữ liệu từ URL: {url}")
    k = k+1
    print(k)
    
    # Đóng trình duyệt sau mỗi lần lặp
    driver.quit()

# Chuyển danh sách dữ liệu thành DataFrame 
output_df = pd.DataFrame(all_data)
print(output_df.count())

# Lưu vào CSV
output_df.to_csv('nha_o_desc_0-100.csv', index=False, encoding='utf-8')
print("done")


Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-nha-dat-quan-ha-dong-ha-noi/119873827.htm#px=SR-stickyad-[PO-1][PL-top]
1
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-nha-dat-quan-long-bien-ha-noi/120606377.htm#px=SR-stickyad-[PO-2][PL-top]
2
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-nha-dat-quan-thanh-xuan-ha-noi/119914033.htm
3
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-nha-dat-quan-dong-da-ha-noi/119987241.htm
4
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-nha-dat-quan-hoang-mai-ha-noi/120691802.htm
5
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-nha-dat-quan-dong-da-ha-noi/120313241.htm
6
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-nha-dat-quan-ba-dinh-ha-noi/120293561.htm
7
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-nha-dat-huyen-gia-lam-ha-noi/120645421.htm
8
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-nha-dat-quan-ba-dinh-ha-noi/119853594.htm
9
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-

**2.2 Căn hộ/ Chung cư**

In [32]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

k=0

# Đọc danh sách URL từ file CSV
input_file = 'chungcu_0_116.csv'  
urls_df = pd.read_csv(input_file)

# Hàm lấy thông tin từ trang
def get_text_or_none(driver, by, identifier):
    try:
        element = driver.find_element(by, identifier)
        return element.text
    except Exception as e:
        return None

# Danh sách để lưu dữ liệu từ mỗi trang
all_data = []

# Vòng lặp qua từng URL để lấy dữ liệu
for url in urls_df['links']:
    # Khởi tạo trình duyệt cho từng lần lặp
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    driver.get(url)
    # time.sleep(2)  # Chờ trang tải xong

    # Nhấn vào nút "Xem thêm" nếu có
    try:
        xem_them_button = driver.find_element(By.XPATH, '//button[contains(@class, "styles_button__SVZnw") and contains(text(), "Xem thêm")]')
        driver.execute_script("arguments[0].click();", xem_them_button)
        # time.sleep(1)  # Đợi một chút để phần còn lại của trang tải xong
    except Exception as e:
        print("Không tìm thấy nút 'Xem thêm' hoặc lỗi khi nhấn:", e)

    # Lấy thông tin từ trang
    data = {
        'Link': url,
        'Địa chỉ': get_text_or_none(driver, By.CLASS_NAME, 'bwq0cbs.flex-1'),
        'Giá tổng': get_text_or_none(driver, By.CSS_SELECTOR, 'b.pyhk1dv'),
        'Giá/m²': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="price_m2"]'),
        'Số phòng ngủ': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="rooms"]'),
        'Số phòng vệ sinh': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="toilets"]'),
        'Giấy tờ pháp lý': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="property_legal_document"]'),
        'Loại hình căn hộ': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="apartment_type"]'),
        'Diện tích': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="size"]'),
        'Hướng cửa chính': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="direction"]'),
        'Hướng ban công': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="balconydirection"]'),
        'Tình trạng nội thất': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="furnishing_sell"]')
    }

    # Thêm dữ liệu vào danh sách
    all_data.append(data)
    print(f"Đã lấy dữ liệu từ URL: {url}")
    k = k+1
    print(k)
    
    # Đóng trình duyệt sau mỗi lần lặp
    driver.quit()

# Chuyển danh sách dữ liệu thành DataFrame 
output_df = pd.DataFrame(all_data)
print(output_df.count())

# Lưu vào CSV
output_df.to_csv('chungcu_desc_raw_0_116.csv', index=False, encoding='utf-8')
print("done")


Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-ba-dinh-ha-noi/120369041.htm#px=SR-stickyad-[PO-1][PL-top]
1
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-dong-da-ha-noi/120551779.htm#px=SR-stickyad-[PO-2][PL-top]
2
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-thanh-xuan-ha-noi/120671736.htm#px=SR-stickyad-[PO-3][PL-top]
3
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-thanh-xuan-ha-noi/120542779.htm#px=SR-stickyad-[PO-4][PL-top]
4
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-can-ho-chung-cu-huyen-gia-lam-ha-noi/118958736.htm
5
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-can-ho-chung-cu-huyen-gia-lam-ha-noi/106928409.htm
6
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-can-ho-chung-cu-huyen-gia-lam-ha-noi/115426307.htm
7
Đã lấy dữ liệu từ URL: https://www.nhatot.com/mua-ban-can-ho-chung-cu-quan-long-bien-ha-noi/120304491.htm
8
Đã lấy dữ liệu từ URL

In [33]:
output_df

,Link,Địa chỉ,Giá tổng,Giá/m²,Số phòng ngủ,Số phòng vệ sinh,Giấy tờ pháp lý,Loại hình căn hộ,Diện tích,Hướng cửa chính,Hướng ban công,Tình trạng nội thất
0,https://www.nhatot.com/mua-ban-can-ho-chung-cu...,"Phố Kim Mã, Phường Ngọc Khánh, Quận Ba Đình, H...","3,28 tỷ","50,46 triệu/m²",2 phòng,None,Sổ hồng riêng,"Tập thể, cư xá",65 m²,None,None,Nội thất đầy đủ
1,https://www.nhatot.com/mua-ban-can-ho-chung-cu...,"Phố Tây Sơn, Phường Quang Trung, Quận Đống Đa,...","1,65 tỷ",33 triệu/m²,2 phòng,1 phòng,Sổ hồng riêng,Chung cư,50 m²,None,None,Nội thất cao cấp
2,https://www.nhatot.com/mua-ban-can-ho-chung-cu...,"Đường Nguyễn Quý Đức, Phường Thanh Xuân Bắc, Q...","2,86 tỷ","42,06 triệu/m²",2 phòng,1 phòng,Sổ hồng riêng,"Tập thể, cư xá",68 m²,None,None,Nội thất đầy đủ
3,https://www.nhatot.com/mua-ban-can-ho-chung-cu...,"Phố Quan Nhân, Phường Nhân Chính, Quận Thanh X...","1,86 tỷ","33,82 triệu/m²",2 phòng,1 phòng,Sổ hồng riêng,"Căn hộ dịch vụ, mini",55 m²,None,None,None
4,https://www.nhatot.com/mua-ban-can-ho-chung-cu...,"Yên Viên, Thị trấn Trâu Quỳ, Huyện Gia Lâm, Hà...","2,5 tỷ","41,67 triệu/m²",2 phòng,1 phòng,Sổ hồng riêng,Chung cư,60 m²,None,None,Nội thất đầy đủ
...,...,...,...,...,...,...,...,...,...,...,...,...
2303,https://www.nhatot.com/mua-ban-can-ho-chung-cu...,"Xuân La, Phường Xuân Tảo, Quận Bắc Từ Liêm, Hà...",10 tỷ,"104,49 triệu/m²",2 phòng,2 phòng,Sổ hồng riêng,Chung cư,95.7 m²,Tây,Đông,Nội thất cao cấp
2304,https://www.nhatot.com/mua-ban-can-ho-chung-cu...,"số 622, Minh Khai, Phường Vĩnh Tuy, Quận Hai B...","5,35 tỷ","72,30 triệu/m²",2 phòng,2 phòng,Sổ hồng riêng,Chung cư,74 m²,Đông,Tây,Nội thất đầy đủ
2305,https://www.nhatot.com/mua-ban-can-ho-chung-cu...,"Đường Lê Đức Thọ, Phường Mỹ Đình 2, Quận Nam T...",28 tỷ,56 triệu/m²,5 phòng,4 phòng,Hợp đồng mua bán,Penthouse,500 m²,None,None,None
2306,https://www.nhatot.com/mua-ban-can-ho-chung-cu...,"Nguyễn Hoàng Tôn, Phường Xuân La, Quận Tây Hồ,...","6,5 tỷ","85,53 triệu/m²",2 phòng,2 phòng,Đang chờ sổ,Chung cư,76 m²,Đông Nam,Tây Bắc,Nội thất cao cấp


**2.3 Đất thổ cư**

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

k=0

# Đọc danh sách URL từ file CSV
input_file = 'chungcu_0_116.csv'  
urls_df = pd.read_csv(input_file)

# Hàm lấy thông tin từ trang
def get_text_or_none(driver, by, identifier):
    try:
        element = driver.find_element(by, identifier)
        return element.text
    except Exception as e:
        return None

# Danh sách để lưu dữ liệu từ mỗi trang
all_data = []

# Vòng lặp qua từng URL để lấy dữ liệu
for url in urls_df['links']:
    # Khởi tạo trình duyệt cho từng lần lặp
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    driver.get(url)
    # time.sleep(2)  # Chờ trang tải xong

    # Nhấn vào nút "Xem thêm" nếu có
    try:
        xem_them_button = driver.find_element(By.XPATH, '//button[contains(@class, "styles_button__SVZnw") and contains(text(), "Xem thêm")]')
        driver.execute_script("arguments[0].click();", xem_them_button)
        # time.sleep(1)  # Đợi một chút để phần còn lại của trang tải xong
    except Exception as e:
        print("Không tìm thấy nút 'Xem thêm' hoặc lỗi khi nhấn:", e)

    # Lấy thông tin từ trang
    data = {
        'Link': url,
        'Địa chỉ': get_text_or_none(driver, By.CLASS_NAME, 'bwq0cbs.flex-1'),
        'Giá tổng': get_text_or_none(driver, By.CSS_SELECTOR, 'b.pyhk1dv'),
        'Giá/m²': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="price_m2"]'),
        'Số phòng ngủ': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="rooms"]'),
        'Số phòng vệ sinh': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="toilets"]'),
        'Giấy tờ pháp lý': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="property_legal_document"]'),
        'Loại hình căn hộ': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="apartment_type"]'),
        'Diện tích': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="size"]'),
        'Hướng cửa chính': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="direction"]'),
        'Hướng ban công': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="balconydirection"]'),
        'Tình trạng nội thất': get_text_or_none(driver, By.CSS_SELECTOR, 'strong[itemprop="furnishing_sell"]')
    }

    # Thêm dữ liệu vào danh sách
    all_data.append(data)
    print(f"Đã lấy dữ liệu từ URL: {url}")
    k = k+1
    print(k)
    
    # Đóng trình duyệt sau mỗi lần lặp
    driver.quit()

# Chuyển danh sách dữ liệu thành DataFrame 
output_df = pd.DataFrame(all_data)
print(output_df.count())

# Lưu vào CSV
output_df.to_csv('chungcu_desc_raw_0_116.csv', index=False, encoding='utf-8')
print("done")


In [ ]:
output_df